In [6]:
import numpy as np
import pandas as pd
import json

In [3]:
def load_json(fname):
    with open(fname, encoding="utf-8") as f:
        json_obj = json.load(f)

    return json_obj

In [17]:
path = "./kakao_arena_melon/arena_data/"
gt_fname = path + "answers/val.json"
rec_fname = path + "results/results.json"

In [18]:
gt_playlists = load_json(gt_fname) # 정답
rec_playlists = load_json(rec_fname) # 추천 결과

In [20]:
gt_playlists[0]

{'tags': ['봄'],
 'id': 126990,
 'plylst_title': 'ㅇㅅㅇ헤헿',
 'songs': [361515,
  67882,
  367963,
  670887,
  115289,
  598537,
  343416,
  449244,
  670842,
  67577,
  284992,
  615912],
 'like_cnt': 1,
 'updt_date': '2017-06-28 12:32:20.000'}

------

> 평가하기 위해 데이터셋 확인
- 플레이리스트 수 비교
- 추천 곡 결과 개수 비교
- 추천 태그 결과 개수 비교
- 한 플레이리스트에 중복된 곡 추천 허용 안됨

In [22]:
# {plylst id : plylst}
gt_dict = {g["id"]: g for g in gt_playlists}
gt_dict[126990]

{'tags': ['봄'],
 'id': 126990,
 'plylst_title': 'ㅇㅅㅇ헤헿',
 'songs': [361515,
  67882,
  367963,
  670887,
  115289,
  598537,
  343416,
  449244,
  670842,
  67577,
  284992,
  615912],
 'like_cnt': 1,
 'updt_date': '2017-06-28 12:32:20.000'}

In [24]:
# 플레이리스트 수 비교
gt_ids = set([g["id"] for g in gt_playlists])
rec_ids = set([r["id"] for r in rec_playlists])

if gt_ids != rec_ids:
    raise Exception("결과의 플레이리스트 수가 올바르지 않습니다.")
else:
    print("플레이리스트 수 일치")

OK


In [29]:
# 각 플레이리스트의 추천 곡, 태그 개수가 100, 10개 인지 확인
rec_song_counts = [len(p["songs"]) for p in rec_playlists] # [100, 100, ...]
rec_tag_counts = [len(p["tags"]) for p in rec_playlists] # [10, 10, ...]

if set(rec_song_counts) != set([100]):
    raise Exception("추천 곡 결과의 개수가 맞지 않습니다.")
else:
    print("추천 곡 결과 수 일치")

if set(rec_tag_counts) != set([10]):
    raise Exception("추천 태그 결과의 개수가 맞지 않습니다.")
else:
    print("추천 태그 결과 수 일치")

추천 곡 결과 수 일치
추천 태그 결과 수 일치


In [33]:
# 각 플레이리스트의 중복된 곡, 태그가 있는 지 확인
rec_unique_song_counts = [len(set(p["songs"])) for p in rec_playlists]
rec_unique_tag_counts = [len(set(p["tags"])) for p in rec_playlists]

if set(rec_unique_song_counts) != set([100]):
    raise Exception("한 플레이리스트에 중복된 곡 추천은 허용되지 않습니다.")
else:
    print("중복 곡 없음")

if set(rec_unique_tag_counts) != set([10]):
    raise Exception("한 플레이리스트에 중복된 태그 추천은 허용되지 않습니다.")
else:
    print("중복 태그 없음")

중복 곡 없음
중복 태그 없음


---

> 평가

### NDCG는 ranking quality를 측정하는 지표이다
- ndcg(normalized Discounted Cumulative Gain) = dcg/idcg 
<br> -- dcg의 정규화된 지표로, 기존의 dcg는 랭킹 결과의 길이인 p가 커질수록 누적합인 dcg는 커지므로 값이 많이 변하므로 p에 상관없이 일정 스케일의 값을 가지도록 정규화가 필요하여 idcg로 나눈 값을 사용한다. 
<br> -- 0 <= ndcg <= 1
<br> -- 해석: 1 에 가까울 수록 좋은 지표이다.

- dcg(Discounted Cumulative Gain)
<br> -- 상위 p개의 추천 결과들의 관련성을 합한 누적값으로, 랭킹 순서에 따라 점점 비중을 줄여 discounted 된 관련도를 계산하는 방법이다. 랭킹 순서에 대한 로그 함수를 분모로 두면, 하위권으로 갈수록 rel 대비 작은 dcg 값을 가지므로 하위관 결과에 패널티를 주는 방식이다.

- idcg(Ideal Discounted Cumulative Gain)
<br> -- rel가 큰 순서대로 재배열한 후 상위 p개를 선택한 rel_opt에 대해서 DCG를 계산한 결과로 선택된 p개 결과로 가질 수 있는 가장 큰 DCG 값

In [43]:
# 평가 함수 정의
def idcgs(l):
    return sum((1.0 / np.log(i + 2) for i in range(l)))

def ndcg(gt, rec):
    dcg = 0.0
    for i, r in enumerate(rec):
        if r in gt:
            dcg += 1.0 / np.log(i + 2)

    return dcg / idcgs(len(gt))

In [44]:
music_ndcg = 0.0
tag_ndcg = 0.0

for rec in rec_playlists:
    # gt: 추천 플리의 정답 결과
    gt = gt_dict[rec["id"]]

    # 정답 곡,태그 리스트와 추천 곡, 태그 리스트(100, 10개)를 ndcg로 계산
    music_ndcg += ndcg(gt["songs"], rec["songs"][:100])
    tag_ndcg += ndcg(gt["tags"], rec["tags"][:10])

music_ndcg, tag_ndcg

(391.83677640995495, 3711.855342228854)

In [ ]:
music_ndcg = music_ndcg / len(rec_playlists)
tag_ndcg = tag_ndcg / len(rec_playlists)